In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# 브라우저 옵션 설정
options = Options()
# options.add_argument("--headless")  # 로그인해야 하므로 꺼둠
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 로그인 및 필터 후 검색 페이지 수동 접속
print("🔐 브라우저가 열립니다. ProQuest에 로그인하고, 필터와 검색어 설정 후 원하는 검색결과 페이지로 이동하세요.")
driver.get("https://www.proquest.com/")
input("✅ 필터 적용 완료 후 Enter를 눌러주세요...")

results = []

MAX_PAGES = 80  # 원하는 페이지 수만큼 반복
for page in range(1, MAX_PAGES + 1):
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.select('div.resultHeader div h3 a')

    print(f"📄 페이지 {page}에서 {len(links)}개 링크 수집")

    for link in links:
        title = link.get_text(strip=True)
        href = link.get('href')
        full_url = "https://www.proquest.com" + href if href.startswith("/") else href

        results.append({
            'title': title,
            'link': full_url
        })

    # 다음 페이지로 이동 시도
    try:
        next_button = driver.find_element(By.XPATH, '//*[@id="updateForm"]/nav/ul/li[9]/a')
        if next_button.is_enabled():
            driver.execute_script("arguments[0].click();", next_button)
            print("➡️ 다음 페이지로 이동...")
            time.sleep(4)
        else:
            print("🚫 다음 페이지 버튼이 비활성화되어 있습니다.")
            break
    except NoSuchElementException:
        print("❌ 다음 페이지 버튼을 찾을 수 없습니다.")
        break
    except ElementClickInterceptedException:
        print("⚠️ 버튼 클릭 실패. 스크롤 후 재시도")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

# CSV 저장
df = pd.DataFrame(results)
df.to_csv("proquest_wallstreet.csv", index=False, encoding="utf-8-sig")
print(f"✅ 완료: 총 {len(df)}개의 항목을 저장했습니다.")
driver.quit()

/opt/anaconda3/envs/Academia-Industry-gap-analysis/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


🔐 브라우저가 열립니다. ProQuest에 로그인하고, 필터와 검색어 설정 후 원하는 검색결과 페이지로 이동하세요.
📄 페이지 1에서 20개 링크 수집
➡️ 다음 페이지로 이동...
📄 페이지 2에서 20개 링크 수집
➡️ 다음 페이지로 이동...
📄 페이지 3에서 20개 링크 수집
➡️ 다음 페이지로 이동...
📄 페이지 4에서 20개 링크 수집
➡️ 다음 페이지로 이동...
📄 페이지 5에서 20개 링크 수집
➡️ 다음 페이지로 이동...
📄 페이지 6에서 20개 링크 수집
➡️ 다음 페이지로 이동...
📄 페이지 7에서 20개 링크 수집
➡️ 다음 페이지로 이동...
📄 페이지 8에서 20개 링크 수집
➡️ 다음 페이지로 이동...
📄 페이지 9에서 4개 링크 수집
❌ 다음 페이지 버튼을 찾을 수 없습니다.
✅ 완료: 총 164개의 항목을 저장했습니다.


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re

# 크롬 옵션 설정
options = Options()
# options.add_argument('--headless')  # 필요 시 주석 해제
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# CSV 불러오기
df = pd.read_csv('proquest_wallstreet.csv')
# df = df.head(20)  # 테스트로 일부만 실행

results = []

for idx, row in df.iterrows():
    url = row['link']
    title = row['title']

    try:
        driver.get(url)
        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # ✅ URL에서 docview ID 추출
        doc_id_match = re.search(r'/docview/(\d+)', url)
        doc_id = doc_id_match.group(1) if doc_id_match else ''
        mstar_id = f"MSTAR_{doc_id}" if doc_id else ''

        # ✅ 본문(Content)
        content = ''
        xpaths_content = [
            f'//*[@id="fulltext_field_{mstar_id}"]/div/root/text/p' if mstar_id else '',
            '//*[@id="fullTextZone"]//p',
            '//*[@id="main-content"]//p',
            '//*[@id="companionColumn-0"]//p',
        ]
        for xp in xpaths_content:
            if not xp:
                continue
            try:
                paragraphs = driver.find_elements(By.XPATH, xp)
                if paragraphs:
                    content = '\n'.join(p.text.strip() for p in paragraphs if p.text.strip())
                    break
            except:
                continue

        # ✅ 날짜(Date): BeautifulSoup로 strong + 형제 텍스트에서 추출
        date_text = ''
        try:
            strong_tag = soup.find('strong', string=lambda s: s and ';' in s)
            if strong_tag and strong_tag.next_sibling:
                raw = strong_tag.next_sibling.strip()
                match = re.search(r'\d{2} \w{3} \d{4}', raw)  # "04 Feb 2025"
                if match:
                    date_text = match.group()
                else:
                    date_text = raw
        except:
            pass

        # ✅ 미디어(Media)
        media = ''
        xpaths_media = [
            f'//*[@id="pubPopoverTrigger-{mstar_id}"]/span' if mstar_id else '',
            '//*[@id="bibSource"]/span',
            '//div[@class="publicationTitle"]',
            '//div[@class="bibSource"]',
            '//span[@class="publicationTitle"]',
        ]
        for xp in xpaths_media:
            if not xp:
                continue
            try:
                media_el = driver.find_element(By.XPATH, xp)
                media = media_el.text.strip().split(';')[0]
                if media:
                    break
            except:
                continue

        results.append({
            'title': title,
            'link': url,
            'date': date_text,
            'media': media,
            'content': content
        })

        print(f"[{idx+1}] ✅ 완료: {title[:40]}...")

    except Exception as e:
        print(f"[{idx+1}] ❌ 실패: {url} → {e}")
        results.append({
            'title': title,
            'link': url,
            'date': '',
            'media': '',
            'content': ''
        })

# 저장
output_df = pd.DataFrame(results, columns=['title', 'link', 'date', 'media', 'content'])
output_df.to_csv('wallstreet_content.csv', index=False, encoding='utf-8-sig')
print("📄 저장 완료: wallstreet_content.csv")

driver.quit()

In [13]:
import pandas as pd

df=pd.read_csv('wallstreet_content.csv')
df.head(20)

,title,link,date,media,content
0,ArtificialIntelligence(A Special Report) --- H...,https://www.proquest.com/docview/3132486081/2B...,25 Nov 2024,Wall Street Journal,The current generation of college students is ...
1,ArtificialIntelligence(A Special Report) --- T...,https://www.proquest.com/docview/3132486071/2B...,25 Nov 2024,Wall Street Journal,ChatGPT is barely two years old. And yet it's ...
2,ArtificialIntelligence(A Special Report) --- F...,https://www.proquest.com/docview/3162691166/2B...,03 Feb 2025,Wall Street Journal,The race for AI dominance launched a stampede ...
3,Crunchbase UsesArtificialIntelligenceTo Predic...,https://www.proquest.com/docview/3168537629/2B...,20 Feb 2025,Wall Street Journal,"Crunchbase, the firm best known for its startu..."
4,On the Clock: Bosses' Mental Fitness Set for A...,https://www.proquest.com/docview/3096788378/2B...,26 Aug 2024,Wall Street Journal,Bosses already live in fear that a verbal miss...
5,EXCHANGE --- The Rundown on Building an AI Sup...,https://www.proquest.com/docview/3191507034/2B...,19 Apr 2025,Wall Street Journal,President Trump enthused on social media this ...
6,ArtificialIntelligence(A Special Report) --- H...,https://www.proquest.com/docview/3132486040/2B...,25 Nov 2024,Wall Street Journal,Artificial intelligence is poised to transform...
7,ArtificialIntelligence(A Special Report) --- H...,https://www.proquest.com/docview/3162691094/2B...,03 Feb 2025,Wall Street Journal,You can't stop an AI chatbot from sometimes ha...
8,The Future of Everything: TheArtificialIntelli...,https://www.proquest.com/docview/3171573768/2B...,27 Feb 2025,Wall Street Journal,"In a large, brightly-lit warehouse in Flowery ..."
9,The Future of Everything: TheArtificialIntelli...,https://www.proquest.com/docview/3098087388/2B...,29 Aug 2024,Wall Street Journal,A plate of food has been set before you -- per...


# 빈 콘텐츠 추가수집 

In [36]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re

# ▶ 기존 결과 불러오기
existing_df = pd.read_csv('wallstreet_content.csv')

# ▶ content가 빈 row만 필터링
empty_content_df = existing_df[existing_df['content'].isna() | (existing_df['content'].str.strip() == '')]

empty_content_df=empty_content_df.head(20)

if empty_content_df.empty:
    print("✅ 모든 content가 이미 수집되어 있습니다.")
else:
    print(f"🔄 수집할 문서 수: {len(empty_content_df)}")

    # ▶ 크롬 옵션 설정
    options = Options()
    # options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')

    # ▶ 드라이버 실행
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    results = []

    for idx, row in empty_content_df.iterrows():
        url = row['link']
        title = row['title']

        try:
            driver.get(url)
            time.sleep(3)

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # ✅ URL에서 docview ID 추출
            doc_id_match = re.search(r'/docview/(\d+)', url)
            doc_id = doc_id_match.group(1) if doc_id_match else ''
            mstar_id = f"MSTAR_{doc_id}" if doc_id else ''

            # ✅ 본문(Content)
            content = ''
            xpaths_content = [
                f'//*[@id="fulltext_field_{mstar_id}"]/div/root/text/p' if mstar_id else '',
                '//*[@id="fullTextZone"]//p',
                '//*[@id="main-content"]//p',
                '//*[@id="companionColumn-0"]//p',
            ]
            for xp in xpaths_content:
                if not xp:
                    continue
                try:
                    paragraphs = driver.find_elements(By.XPATH, xp)
                    if paragraphs:
                        content = '\n'.join(p.text.strip() for p in paragraphs if p.text.strip())
                        break
                except:
                    continue

            # ✅ 날짜(Date): BeautifulSoup로 strong + 형제 텍스트에서 추출
            date_text = row['date']  # 기본값: 기존 값 유지
            try:
                strong_tag = soup.find('strong', string=lambda s: s and ';' in s)
                if strong_tag and strong_tag.next_sibling:
                    raw = strong_tag.next_sibling.strip()
                    match = re.search(r'\d{2} \w{3} \d{4}', raw)
                    if match:
                        date_text = match.group()
                    else:
                        date_text = raw
            except:
                pass

            # ✅ 미디어(Media)
            media = row['media']  # 기본값: 기존 값 유지
            xpaths_media = [
                f'//*[@id="pubPopoverTrigger-{mstar_id}"]/span' if mstar_id else '',
                '//*[@id="bibSource"]/span',
                '//div[@class="publicationTitle"]',
                '//div[@class="bibSource"]',
                '//span[@class="publicationTitle"]',
            ]
            for xp in xpaths_media:
                if not xp:
                    continue
                try:
                    media_el = driver.find_element(By.XPATH, xp)
                    media = media_el.text.strip().split(';')[0]
                    if media:
                        break
                except:
                    continue

            # ✅ 결과 저장
            existing_df.loc[idx, 'content'] = content
            existing_df.loc[idx, 'date'] = date_text
            existing_df.loc[idx, 'media'] = media

            print(f"[{idx+1}] ✅ 업데이트 완료: {title[:40]}...")

        except Exception as e:
            print(f"[{idx+1}] ❌ 실패: {url} → {e}")

    # ▶ CSV 덮어쓰기 저장
    existing_df.to_csv('wallstreet_content.csv', index=False, encoding='utf-8-sig')
    print("📄 저장 완료: wallstreet_content.csv")

    driver.quit()

🔄 수집할 문서 수: 20
[357] ✅ 업데이트 완료: EXCHANGE --- Markets & Finance: Aflac Sa...
[358] ✅ 업데이트 완료: OpenAI Aims for $90 Billion Value With S...
[359] ✅ 업데이트 완료: Microsoft CEO Says Google's Tactics Hurt...
[360] ✅ 업데이트 완료: Business & Finance...
[361] ✅ 업데이트 완료: U.S. News: FBI Investigates Chief-of-Sta...
[362] ✅ 업데이트 완료: Apple to Source More Phones From India -...
[363] ✅ 업데이트 완료: Microsoft To Invest In U.A.E. AI Firm...
[364] ✅ 업데이트 완료: World News: Lebanon Takes Steps to Loose...
[396] ✅ 업데이트 완료: Business & Finance...
[397] ✅ 업데이트 완료: Business & Finance...
[398] ✅ 업데이트 완료: Business & Finance...
[399] ✅ 업데이트 완료: Business & Finance...
[400] ✅ 업데이트 완료: Business & Finance...
[401] ✅ 업데이트 완료: Markets: Stock Spotlight...
[402] ✅ 업데이트 완료: Former Binance.US CEO Has New Firm...
[403] ✅ 업데이트 완료: Markets: Stock Spotlight...
[404] ✅ 업데이트 완료: Markets & Finance: Stock Spotlight...
[405] ✅ 업데이트 완료: Business & Finance...
[406] ✅ 업데이트 완료: Business & Finance...
[407] ✅ 업데이트 완료: Markets: Stock Spotlight...
📄 저장

In [38]:
import pandas as pd
test=pd.read_csv('wallstreet_content.csv')
test.info() #기존 962개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1593 entries, 0 to 1592
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1593 non-null   object
 1   link     1593 non-null   object
 2   date     962 non-null    object
 3   media    962 non-null    object
 4   content  962 non-null    object
dtypes: object(5)
memory usage: 62.4+ KB
